In [22]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import warnings

import sqlite3  # Para trabajar con SQLite
import pandas as pd  # Para manejar datos como DataFrames
import duckdb  # Para trabajar con DuckDB (base de datos en memoria)
import pandas as pd  # Para trabajar con DataFrames
import pandas as pd  # Para manejar datos como DataFrames
from tabulate import tabulate




warnings.filterwarnings("ignore", category=RuntimeWarning)
# ---------- Paths ----------
path = Path(r"E:/TFG/Datos/Importants/")
data1 = path / "HadISST_sst_v1.1_196001_202105.nc"
data2 = path / "sst.mnmean_v5_196001_202105.nc"
NT = 732  # meses a usar (como en tu notebook)
OUT = path / "_sql_exports"
OUT.mkdir(exist_ok=True)

In [2]:
# ---------- Funciones de carga y procesamiento ----------
def to_lat_lon_latitude_longitude(da: xr.DataArray) -> xr.DataArray:
    """Convierte las coordenadas de latitud y longitud a convenciones estándar."""
    
    # Renombrar "latitude" a "lat" si existe
    if "latitude" in da.coords and "lat" not in da.coords:
        da = da.rename({"latitude": "lat"})
        
    # Renombrar "longitude" a "lon" si existe
    if "longitude" in da.coords and "lon" not in da.coords:
        da = da.rename({"longitude": "lon"})
        
    # Convierte las longitudes a la convención -180...180
    lon = da["lon"]
    lon2 = ((lon + 180) % 360) - 180
    da = da.assign_coords(lon=lon2).sortby("lon")
    
    return da

    


def open_sst_clean(fp: Path, varname: str, nt: int = NT) -> xr.DataArray:
    """Abre el dataset NetCDF y limpia con xarray."""
    ds = xr.open_dataset(fp, decode_cf=True, mask_and_scale=True)
    ds = to_lat_lon_latitude_longitude(ds)
    if varname not in ds:
        raise KeyError(f"{varname} no está en {fp.name}. vars={list(ds.data_vars)}")
    
    
    # Seleccionamos la variable de interés (por ejemplo, "sst") y recortamos el tiempo
    da = ds[varname].isel(time=slice(0, nt))

    # Aplicamos el filtro de valores válidos para SST
    da = da.where((da >= -1.79) & (da <= 35))  # Reemplazar valores fuera del rango con NaN

    # Aseguramos que las longitudes estén en la convención -180 a 180
    return da


# Abre el Dataset completo para cada archivo


def to_float_nan(a):
    """Convierte Variable/MaskedArray a ndarray float con NaNs donde haya máscara."""
    if np.ma.isMaskedArray(a):
        return np.ma.filled(a, np.nan).astype(float)
    return np.array(a, dtype=float)

def calc_monthly_anoms(fen, detrend=True):
    """
    fen: (time, lat, lon) mensual (DataArray)
    ds: Dataset completo (contiene las coordenadas)
    return: anoms (year, month, lat, lon)
    """
    arr = to_float_nan(fen)  # Aseguramos que los valores NaN se convierten a np.nan


    # Si viniera 4D (time, level, lat, lon), toma level 0
    if arr.ndim == 4:
        arr = arr[:, 0, :, :]
    if arr.ndim != 3:
        raise ValueError(f"Se esperaba (time, lat, lon), pero llegó {arr.shape}")

    nt, ny, nx = arr.shape
    nyr = nt // 12
    nt_use = nyr * 12
    dat = arr[:nt_use].copy()

    # Detrend (ajustamos la tendencia temporal si se pide)
    if detrend:
        x = np.arange(nt_use)
        X = np.column_stack([x, np.ones(nt_use)])  # Matriz de diseño (tiempo, 1) para la regresión
        Y = dat.reshape(nt_use, -1)  # Aplanamos el array para usar mínimos cuadrados
        Yf = np.nan_to_num(Y, nan=0.0)  # Convertir NaNs a 0 para la regresión

        coeffs = np.linalg.lstsq(X, Yf, rcond=None)[0]  # Resuelve la regresión lineal
        trend = (X @ coeffs).reshape(dat.shape)  # Calcula la tendencia en el tiempo

        mask = np.isfinite(dat)  # Creamos una máscara de los valores no nulos
        dat[mask] = dat[mask] - trend[mask]  # Restamos la tendencia a los datos no nulos

    # Reorganizar datos a (año, mes, lat, lon)
    dat4 = dat.reshape(nyr, 12, ny, nx)  # (year, month, lat, lon)

    # Calcular la climatología mensual (media de cada mes a lo largo de los años)
    clim = np.nanmean(dat4, axis=0)  # Calculamos la climatología mensual

    # Anomalías: restamos la climatología mensual
    anoms = dat4 - clim[None, :, :, :]  # (year, month, lat, lon)

    # Acceder a las coordenadas lat y lon directamente
    lat = fen.coords["lat"]
    lon = fen.coords["lon"]

    # Devolvemos el DataArray con las anomalías
    return xr.DataArray(
        anoms,
        dims=["year", "month", "lat", "lon"],  # Usamos "lat" y "lon" directamente
        coords={
            "year": np.arange(nyr),
            "month": np.arange(12),
            "lat": lat,  # Usamos las coordenadas de latitud
            "lon": lon,  # Usamos las coordenadas de longitud
        },
    )

def region_mean(anoms: xr.DataArray, region: dict) -> xr.DataArray:
    """Calcula la media de las anomalías para una región específica."""
    # Seleccionar la región usando las coordenadas de latitud y longitud
    anoms_region = anoms.sel(lat=region["lat"], lon=region["lon"])  # Seleccionamos la región

    # Calculamos la media espacial sobre las dimensiones de latitud y longitud
    return anoms_region.mean(dim=["lat", "lon"], skipna=True)  # Media sobre latitudes y longitudes






In [12]:
# Cargar los datos y calcular las anomalías mensuales
had_sst = open_sst_clean(data1, "sst", NT)
ers_sst = open_sst_clean(data2, "sst", NT)

# Seleccionar las regiones Niño3.4 y ATL3 y calcular anomalías
NINO34 = dict(lat=slice(6, -6), lon=slice(-170, -120))
ATL3 = dict(lat=slice(4, -4), lon=slice(-20, 0))

# Calcular las anomalías mensuales para Niño3.4 y ATL3
sst_nino34_anoms = calc_monthly_anoms(had_sst)
sst_atl3_anoms = calc_monthly_anoms(ers_sst)

# Calcular la media por región para Niño 3.4 en ambos datasets
nino34_mean_had = region_mean(sst_nino34_anoms, NINO34)
nino34_mean_ers = region_mean(sst_atl3_anoms, NINO34)

# Calcular la media por región para ATL3 en ambos datasets
atl3_mean_had = region_mean(sst_nino34_anoms, ATL3)
atl3_mean_ers = region_mean(sst_atl3_anoms, ATL3)

# Convertir a DataFrame y añadir etiquetas
nino34_df_had = nino34_mean_had.to_dataframe(name="anomaly").reset_index()
nino34_df_had["region"] = "Nino34"
nino34_df_had["source"] = "HadISST_sst"

nino34_df_ers = nino34_mean_ers.to_dataframe(name="anomaly").reset_index()
nino34_df_ers["region"] = "Nino34"
nino34_df_ers["source"] = "ERSST_sst"

atl3_df_had = atl3_mean_had.to_dataframe(name="anomaly").reset_index()
atl3_df_had["region"] = "ATL3"
atl3_df_had["source"] = "HadISST_sst"

atl3_df_ers = atl3_mean_ers.to_dataframe(name="anomaly").reset_index()
atl3_df_ers["region"] = "ATL3"
atl3_df_ers["source"] = "ERSST_sst"

# Unir todas las tablas
final_table = pd.concat([nino34_df_had, nino34_df_ers, atl3_df_had, atl3_df_ers], ignore_index=True)
# Asegurarse que las fechas estén correctas
final_table["year"] = final_table["year"] + 1960  # Añadir el año base para cada observación
final_table["month"] = final_table["month"] + 1  # Asignar correctamente el mes
#


# Guardar la tabla como Parquet
regions_parquet = OUT / "regions_timeseries_anomalies.parquet"




## 1. FROM / JOIN  
Selecciona las tablas y realiza los joins si es necesario. `FROM` especifica las tablas, y los `JOINs` combinan las filas de las tablas relacionadas.  
**Ejemplo:**  
FROM employees e  
JOIN departments d ON e.department_id = d.department_id  

---
## 2. WHERE  
Filtra las filas seleccionadas según las condiciones especificadas.  
**Ejemplo:**  
WHERE salary > 50000  

---
## 3. GROUP BY  
Agrupa las filas por una o varias columnas, útil para funciones agregadas como `COUNT()`, `AVG()`, `MIN()`, `MAX()`, etc.  
**Ejemplo:**  
GROUP BY department_id  

---
## 4. HAVING  
Filtra los grupos resultantes después de un `GROUP BY`, similar al `WHERE` pero para grupos.  
**Ejemplo:**  
HAVING COUNT(*) > 5  

---
## 5. SELECT  
Selecciona las columnas o expresiones para mostrar en el resultado.  
**Ejemplo:**  
SELECT department_id, AVG(salary)  

---
## 6. ORDER BY  
Ordena el resultado final por una o más columnas, de manera ascendente (`ASC`) o descendente (`DESC`).  
**Ejemplo:**  

ORDER BY department_id DESC  

---
## 7. LIMIT / OFFSET  
Restringe el número de filas con `LIMIT` o salta filas con `OFFSET`.  
**Ejemplo:**  

LIMIT 10  

---
Ejemplo completo con todo el orden lógico:

```sql
SELECT department_id, AVG(salary) AS avg_salary  
FROM employees e  
JOIN departments d ON e.department_id = d.department_id  
WHERE salary > 50000  
GROUP BY department_id  
HAVING COUNT(*) > 5  
ORDER BY avg_salary DESC  
LIMIT 10;


## El orden correcto de las cláusulas al escribir una consulta SQL:

1. **SELEECT**: Primero seleccionas las columnas que quieres ver.
2. **FROM**: Luego, debes especificar las tablas o vistas desde las cuales vas a obtener los datos.
3. **WHERE**: Después puedes aplicar filtros para reducir los datos a los que cumplen ciertas condiciones.
4. **GROUP BY**: Si necesitas agrupar los datos (para funciones agregadas como `COUNT()`, `AVG()`, etc.), debe ir después de `WHERE`.
5. **HAVING**: Si necesitas filtrar los resultados de una agrupación (después de `GROUP BY`), se usa `HAVING`.
6. **ORDER BY**: A continuación, puedes ordenar los resultados de la consulta según ciertas columnas.
7. **LIMIT / OFFSET**: Finalmente, puedes usar `LIMIT` para limitar el número de resultados y `OFFSET` para saltarte un número específico de filas.

---

## Resumen del orden lógico de SQL:

1. **FROM / JOIN**: Seleccionar las tablas y hacer joins.
2. **WHERE**: Filtrar las filas según las condiciones.
3. **GROUP BY**: Agrupar las filas.
4. **HAVING**: Filtrar los grupos después del `GROUP BY`.
5. **SELECT**: Seleccionar las columnas o expresiones.
6. **ORDER BY**: Ordenar los resultados.
7. **LIMIT / OFFSET**: Limitar o saltar filas.



In [27]:
headers = ["Year", "Month", "Anomaly", "Region", "Source"]
print(tabulate(final_table.head(10), headers=headers, tablefmt="fancy_grid"))


╒════╤════════╤═════════╤════════════╤══════════╤═════════════╕
│    │   Year │   Month │    Anomaly │ Region   │ Source      │
╞════╪════════╪═════════╪════════════╪══════════╪═════════════╡
│  0 │   1960 │       1 │  0.121873  │ Nino34   │ HadISST_sst │
├────┼────────┼─────────┼────────────┼──────────┼─────────────┤
│  1 │   1960 │       2 │ -0.139709  │ Nino34   │ HadISST_sst │
├────┼────────┼─────────┼────────────┼──────────┼─────────────┤
│  2 │   1960 │       3 │  0.0510608 │ Nino34   │ HadISST_sst │
├────┼────────┼─────────┼────────────┼──────────┼─────────────┤
│  3 │   1960 │       4 │  0.143723  │ Nino34   │ HadISST_sst │
├────┼────────┼─────────┼────────────┼──────────┼─────────────┤
│  4 │   1960 │       5 │  0.20901   │ Nino34   │ HadISST_sst │
├────┼────────┼─────────┼────────────┼──────────┼─────────────┤
│  5 │   1960 │       6 │ -0.0820815 │ Nino34   │ HadISST_sst │
├────┼────────┼─────────┼────────────┼──────────┼─────────────┤
│  6 │   1960 │       7 │ -0.109685  │ N

In [25]:
con = duckdb.connect() 
#con.close() al final de todas las consultas
# Cargar la tabla Parquet en DuckDB
parquet_file = OUT / "regions_timeseries_anomalies.parquet"

# Leer la tabla Parquet y cargarla en DuckDB
con.execute(f"CREATE OR REPLACE TABLE regions_anomalies AS SELECT * FROM read_parquet('{parquet_file}')")

#Consulta SQL para ver las primeras filas
result = con.execute("SELECT * FROM regions_anomalies LIMIT 5").fetchall()
headers = ["Year", "Month", "Anomaly", "Region", "Source"]
print(tabulate(result, headers=headers, tablefmt="fancy_grid"))


╒════════╤═════════╤════════════╤══════════╤═════════════╕
│   Year │   Month │    Anomaly │ Region   │ Source      │
╞════════╪═════════╪════════════╪══════════╪═════════════╡
│   1960 │       1 │  0.121873  │ Nino34   │ HadISST_sst │
├────────┼─────────┼────────────┼──────────┼─────────────┤
│   1960 │       2 │ -0.139709  │ Nino34   │ HadISST_sst │
├────────┼─────────┼────────────┼──────────┼─────────────┤
│   1960 │       3 │  0.0510608 │ Nino34   │ HadISST_sst │
├────────┼─────────┼────────────┼──────────┼─────────────┤
│   1960 │       4 │  0.143723  │ Nino34   │ HadISST_sst │
├────────┼─────────┼────────────┼──────────┼─────────────┤
│   1960 │       5 │  0.20901   │ Nino34   │ HadISST_sst │
╘════════╧═════════╧════════════╧══════════╧═════════════╛


In [31]:
#Consulta SQL anomalías maximas y minimas por region y fuente
result = con.execute("""
SELECT region, source, MIN(anomaly) AS min_anomaly, MAX(anomaly) AS max_anomaly FROM regions_anomalies
GROUP BY region, source
ORDER BY region, source;
""").fetchall()
# Imprimir el resultado con tabulate
headers = ["Region", "Source", "Min Anomaly", "Max Anomaly"]
print(tabulate(result, headers=headers, tablefmt="fancy_grid"))



╒══════════╤═════════════╤═══════════════╤═══════════════╕
│ Region   │ Source      │   Min Anomaly │   Max Anomaly │
╞══════════╪═════════════╪═══════════════╪═══════════════╡
│ ATL3     │ ERSST_sst   │      -1.17767 │       1.3833  │
├──────────┼─────────────┼───────────────┼───────────────┤
│ ATL3     │ HadISST_sst │      -1.2984  │       1.56751 │
├──────────┼─────────────┼───────────────┼───────────────┤
│ Nino34   │ ERSST_sst   │      -1.8694  │       2.35831 │
├──────────┼─────────────┼───────────────┼───────────────┤
│ Nino34   │ HadISST_sst │      -2.05214 │       2.38863 │
╘══════════╧═════════════╧═══════════════╧═══════════════╛


In [33]:
#Consulta SQL de regiones con anomalias mayores a 0.4, ordenadas por año y mes
result = con.execute("""
SELECT * FROM regions_anomalies
WHERE anomaly > 0.4
ORDER BY year, month;
""").fetchall()
headers = ["Year", "Month", "Anomaly", "Region", "Source"]
print(tabulate(result, headers=headers, tablefmt="fancy_grid"))

╒════════╤═════════╤═══════════╤══════════╤═════════════╕
│   Year │   Month │   Anomaly │ Region   │ Source      │
╞════════╪═════════╪═══════════╪══════════╪═════════════╡
│   1960 │       9 │  0.472581 │ ATL3     │ HadISST_sst │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1960 │      10 │  0.470851 │ ATL3     │ HadISST_sst │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1960 │      11 │  0.741458 │ ATL3     │ HadISST_sst │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1960 │      11 │  0.653823 │ ATL3     │ ERSST_sst   │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1960 │      12 │  0.55827  │ ATL3     │ HadISST_sst │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1960 │      12 │  0.543764 │ ATL3     │ ERSST_sst   │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1961 │       1 │  0.400321 │ ATL3     │ HadISST_sst │
├────────┼─────────┼───────────┼──────────┼─────────────┤
│   1961 │    

In [61]:
result = con.execute("""
WITH ConsecutiveMonths AS (
    -- 1. Identificamos meses consecutivos con anomalía >= 0.4 en ATL3
    SELECT 
        year, 
        month, 
        source, 
        anomaly,
        LEAD(month, 1) OVER (PARTITION BY year, source ORDER BY month) AS next_month,
        LEAD(month, 2) OVER (PARTITION BY year, source ORDER BY month) AS next_next_month
    FROM regions_anomalies
    WHERE region = 'ATL3' AND anomaly >= 0.4
),
EventosFiltrados AS (
    -- 2. Filtramos la racha que incluye Julio y calculamos su media por fuente/año
    SELECT 
        year,
        source,
        AVG(anomaly) AS val_anomaly
    FROM ConsecutiveMonths
    WHERE 
        next_month = month + 1 AND next_next_month = month + 2
        AND (month = 7 OR next_month = 7 OR next_next_month = 7)
        AND (month = 6 OR month = 7)
    GROUP BY year, source
)
-- 3. PIVOTADO: Una fila por año, fuentes en columnas con iconos
SELECT 
    year AS Year,
    MAX(CASE WHEN source = 'ERSST_sst' THEN 'YES' ELSE 'NO' END) AS ERSST,
    MAX(CASE WHEN source = 'HadISST_sst' THEN 'YES' ELSE 'NO' END) AS HadISST,
    ROUND(AVG(val_anomaly), 5) AS Avg_Anomaly
FROM EventosFiltrados
GROUP BY year
ORDER BY year;
""").fetchall()

headers = ["Year", "ERSST", "HadISST", "Avg Anomaly"]
# Usamos colalign para que los iconos queden centrados y se vea más ordenado
print(tabulate(result, headers=headers, tablefmt="fancy_grid", colalign=("left", "center", "center", "right")))


╒════════╤═════════╤═══════════╤═══════════════╕
│ Year   │  ERSST  │  HadISST  │   Avg Anomaly │
╞════════╪═════════╪═══════════╪═══════════════╡
│ 1963   │   YES   │    YES    │       1.30211 │
├────────┼─────────┼───────────┼───────────────┤
│ 1968   │   YES   │    YES    │       1.11129 │
├────────┼─────────┼───────────┼───────────────┤
│ 1973   │   NO    │    YES    │       0.40779 │
├────────┼─────────┼───────────┼───────────────┤
│ 1984   │   YES   │    YES    │       0.47758 │
├────────┼─────────┼───────────┼───────────────┤
│ 1987   │   YES   │    YES    │       0.80179 │
├────────┼─────────┼───────────┼───────────────┤
│ 1988   │   YES   │    YES    │       0.86685 │
├────────┼─────────┼───────────┼───────────────┤
│ 1989   │   YES   │    NO     │       0.45219 │
├────────┼─────────┼───────────┼───────────────┤
│ 1999   │   NO    │    YES    │       0.55003 │
├────────┼─────────┼───────────┼───────────────┤
│ 2008   │   NO    │    YES    │       0.54477 │
╘════════╧═════════╧

In [58]:
# Consulta definitiva con lógica de pivotado numérica para evitar errores de orden alfabético
result = con.execute("""
WITH Trimestres AS (
    -- 1. Calculamos la media móvil (ONI) de 3 meses
    SELECT 
        year, month, source, anomaly,
        (anomaly + 
         LEAD(anomaly, 1) OVER (PARTITION BY source ORDER BY year, month) + 
         LEAD(anomaly, 2) OVER (PARTITION BY source ORDER BY year, month)
        ) / 3.0 AS ONI_avg,
        LEAD(year * 12 + month, 1) OVER (PARTITION BY source ORDER BY year, month) AS next_1,
        LEAD(year * 12 + month, 2) OVER (PARTITION BY source ORDER BY year, month) AS next_2
    FROM regions_anomalies
    WHERE region = 'Nino34'
),
Filtrado AS (
    -- 2. Filtramos por umbral de 0.5 y meses consecutivos
    SELECT * FROM Trimestres
    WHERE ONI_avg >= 0.5 
    AND next_1 = (year * 12 + month) + 1
    AND next_2 = (year * 12 + month) + 2
),
Islas AS (
    -- 3. Identificamos rachas (Gaps & Islands)
    SELECT 
        year, month, source, anomaly,
        (year * 12 + month) - ROW_NUMBER() OVER (PARTITION BY source ORDER BY year, month) AS island_id
    FROM Filtrado
),
Eventos AS (
    -- 4. Agrupamos por evento y fuente
    SELECT 
        source,
        MIN(year) AS start_year,
        STRING_AGG(DISTINCT (year::TEXT), ', ' ORDER BY (year::TEXT)) AS Years_List,
        MAX(year) > MIN(year) AS interanual
    FROM Islas
    GROUP BY source, island_id
    HAVING COUNT(*) >= 5 AND interanual
),
Anomalias_Inicio AS (
    -- Obtenemos las anomalías de Nov-Dic para el año de inicio de cada fuente
    SELECT 
        r.source, 
        r.year, 
        AVG(r.anomaly) AS valor_nd
    FROM regions_anomalies r
    WHERE r.region = 'Nino34' AND r.month IN (11, 12)
    GROUP BY r.source, r.year
),
Pivotado AS (
    -- 5. Pivotamos usando 1 y 0 para que MAX() funcione correctamente
    SELECT 
        e.start_year,
        MAX(e.Years_List) AS period,
        MAX(CASE WHEN e.source = 'ERSST_sst' THEN 1 ELSE 0 END) AS has_ersst,
        MAX(CASE WHEN e.source = 'HadISST_sst' THEN 1 ELSE 0 END) AS has_hadisst,
        AVG(a.valor_nd) AS avg_anom
    FROM Eventos e
    LEFT JOIN Anomalias_Inicio a ON e.source = a.source AND e.start_year = a.year
    GROUP BY e.start_year
)
-- 6. Transformación final a iconos
SELECT 
    period AS "Years (Period)",
    CASE WHEN has_ersst = 1 THEN 'YES' ELSE 'NO' END AS ERSST,
    CASE WHEN has_hadisst = 1 THEN 'YES' ELSE 'NO' END AS HadISST,
    ROUND(avg_anom, 5) AS "Avg Nov-Dic Anomaly"
FROM Pivotado
ORDER BY start_year;
""").fetchall()

headers = ["Years (Period)", "ERSST", "HadISST", "Avg Nov-Dic Anomaly"]

print(tabulate(result, headers=headers, tablefmt="fancy_grid"))

╒══════════════════╤═════════╤═══════════╤═══════════════════════╕
│ Years (Period)   │ ERSST   │ HadISST   │   Avg Nov-Dic Anomaly │
╞══════════════════╪═════════╪═══════════╪═══════════════════════╡
│ 1963, 1964       │ YES     │ NO        │               1.07848 │
├──────────────────┼─────────┼───────────┼───────────────────────┤
│ 1965, 1966       │ YES     │ YES       │               1.50915 │
├──────────────────┼─────────┼───────────┼───────────────────────┤
│ 1968, 1969       │ YES     │ YES       │               0.75783 │
├──────────────────┼─────────┼───────────┼───────────────────────┤
│ 1972, 1973       │ YES     │ YES       │               1.95756 │
├──────────────────┼─────────┼───────────┼───────────────────────┤
│ 1976, 1977       │ YES     │ YES       │               0.72091 │
├──────────────────┼─────────┼───────────┼───────────────────────┤
│ 1982, 1983       │ YES     │ YES       │               2.01934 │
├──────────────────┼─────────┼───────────┼────────────────────